In [102]:

import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import pickle
import collections

def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in nltk.word_tokenize(sent)]

path="data/coqa-dev.json"


with open(path,"r")as f:
    train=json.load(f)
    
question_0=collections.Counter()
question_1=collections.Counter()
qa_list=[]
total=0

for paragraph in tqdm(train["data"]):
    context_text=paragraph["story"]
    for i in range(len(paragraph["questions"])):
        question_dict=paragraph["questions"][i]
        answer_dict=paragraph["answers"][i]
        question_text=question_dict["input_text"]
        question=tokenize(question_text)
        answer_text=answer_dict["input_text"]
        sentence_text=answer_dict["span_text"]
        qa_list.append((question_text,answer_text,sentence_text,context_text))
        total+=1
        
        if len(question)>0:
            question_0[question[0]]+=1
            if len(question)>1:
                question_1[" ".join(question[0:2])]+=1

        


100%|██████████| 500/500 [00:00<00:00, 527.06it/s]


In [105]:
for qa in qa_list[1100:1700]:
    q=qa[0]
    print(q)

Why?
Did I think pirates are cool?
Who might be coming up the stairs?
What will tonight be?
Who went to the farm?
Why?
For what?
How many cows did he see there?
Did they have spots?
What color?
What were they doing?
Where?
How did the spots look to him?
Did he move closer?
What leading female character is flying off?
Who was she with?
Who was she leaving?
Was he okay?
Why not?
What else?
What famous talk show host's wife is joining the show?
Who is her wife?
What other show did she star in?
When is Scandal returning?
At what time?
What previous star was jailed?
Where at?
Whose hands does D.C. lie in once she leaves?
What agency does he run?
Who were they waiting for?
Who else?
Who was two hours late?
Where would the carrier go after leaving them?
How was the weather?
What was the employee's name?
Who was his boss?
What were they doing?
Are trees bountiful in that region?
Was this an easy job?
What would they cut with?
What was approaching?
What time of day was it becoming?
And what had

In [100]:
import json
path="data/coqa-dev.json"
with open(path)as f:
    text=json.load(f)
    
text=json.dumps(text,indent=4)
print(text[0:10000])

{
    "version": "1.0",
    "data": [
        {
            "source": "mctest",
            "id": "3dr23u6we5exclen4th8uq9rb42tel",
            "filename": "mc160.test.41",
            "story": "Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. \n\n\"What are you doing, C

In [11]:
import json
path="data/squad-dev-v2.0.json"
with open(path)as f:
    text=json.load(f)
    
text=json.dumps(text,indent=4)
print(text[0:10000])

{
    "version": "v2.0",
    "data": [
        {
            "title": "Normans",
            "paragraphs": [
                {
                    "qas": [
                        {
                            "question": "In what country is Normandy located?",
                            "id": "56ddde6b9a695914005b9628",
                            "answers": [
                                {
                                    "text": "France",
                                    "answer_start": 159
                                },
                                {
                                    "text": "France",
                                    "answer_start": 159
                                },
                                {
                                    "text": "France",
                                    "answer_start": 159
                                },
                                {
                                    "text": "France",
          

In [30]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
from sklearn.datasets import fetch_20newsgroups


x=[["a","b","c"],["a","b","c","d"]]
corpus=["I have a dog which is not a cat .","I have one dog ."]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())

print(X.toarray())

['cat', 'dog', 'have', 'is', 'not', 'one', 'which']
[[1 1 1 1 1 0 1]
 [0 1 1 0 0 1 0]]


In [94]:
import os
import sys
sys.path.append("../")
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import collections

#sentenceを受け取り、tokenizeして返す
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in word_tokenize(sent)]

def answer_find(context_text,answer_start,answer_end,answer_replace):
    context=sent_tokenize(context_text)
    current_p=0
    for i,sent in enumerate(context):
        end_p=current_p+len(sent)
        if current_p<=answer_start and answer_start<=end_p:
            sent_start_id=i
        if current_p<=answer_end and answer_end<=end_p:
            sent_end_id=i
        current_p+=len(sent)+1#スペースが消えている分の追加、end_pの計算のところでするべきかは不明
    answer_sent=" ".join(context[sent_start_id:sent_end_id+1])
    #ここで答えを置換する方法。ピリオドが消滅した場合などに危険なので止める。
    """
    if answer_replace:
        context_text=context_text.replace(context[answer_start:answer_end],"<answer_word>")
        answer_sent=sent_tokenize(context_text)[sent_start_id]
    """
    return sent_start_id


def data_process(input_path,src_path,tgt_path,question_modify,train=True,sub=False,no_answer=False):
    with open(input_path,"r") as f:
        data=json.load(f)
    contexts=[]
    questions=[]
    answer_starts=[]
    answer_ends=[]
    answer_texts=[]
    answers=[]
    sentences=[]
    ids=[]
    answer_replace=False
    cq_list=[]
    for topic in tqdm(data["data"]):
        topic=topic["paragraphs"]
        for paragraph in topic:
            context_text=paragraph["context"].lower()
            for qas in paragraph["qas"]:
                question_text=qas["question"].lower()
                if len(qas["answers"])==0:
                    continue
                a=qas["answers"][0]
                answer_text=a["text"].lower()
                answer_start=a["answer_start"]
                answer_end=a["answer_start"]+len(a["text"])
                context_text=" ".join(tokenize(context_text))
                question_text=" ".join(tokenize(question_text))
                answer_index=answer_find(context_text,answer_start,answer_end,answer_replace)
                cq_list.append((context_text,question_text,answer_index))
    return cq_list
                
        
        
version="2.0"
type="interro_answer_sub"
question_modify=True
question_interro=True

cq_list=data_process(input_path="data/squad-dev-v{}.json".format(version),
            src_path="data/squad-src-dev-{}.txt".format(type),
            tgt_path="data/squad-tgt-dev-{}.txt".format(type),
            question_modify=True,
            train=False,
            sub=False,
            no_answer=False
            )

100%|██████████| 35/35 [00:08<00:00,  3.95it/s]


In [143]:
#SQuADのデータ処理
#必要条件:CoreNLP
#Tools/core...で
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

import os
import sys
sys.path.append("../")
import json
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import collections

def head_find(tgt):
    q_head=["what","how","who","when","which","where","why","whose","whom","is","are","was","were","do","did","does"]
    tgt_tokens=word_tokenize(tgt)
    true_head="<none>"
    for h in q_head:
        if h in tgt_tokens:
            true_head=h
            break
    return true_head

def modify(sentence,question_interro,answer,answer_replace):
    #head=head_find(question)
    """
    if answer in sentence:
        sentence=sentence.replace(answer," ans_rep_tag ")
    """
    sentence=" ".join([sentence,"ans_pos_tag",answer,"interro_tag",question_interro])
    return sentence


def c2wpointer(context_text,context,answer_start,answer_end):#answer_start,endをchara単位からword単位へ変換
    #nltk.tokenizeを使って分割
    #ダブルクオテーションがなぜか変化するので処理
    token_id={}
    cur_id=0
    for i,token in enumerate(context):
        start=context_text.find(token,cur_id)
        token_id[i]=(start,start+len(token))
        cur_id=start+len(token)
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_start and answer_start<=token_id[i][1]:
            answer_start_w=i
            break
    for i in range(len(token_id)):
        if token_id[i][0]<=answer_end and answer_end<=token_id[i][1]:
            answer_end_w=i
            break
    return answer_start_w,answer_end_w

#sentenceを受け取り、tokenizeして返す
def tokenize(sent):
    return [token.replace('``','"').replace("''",'"') for token in word_tokenize(sent)]

#context_textを文分割して、answer_start~answer_end(char単位)のスパンが含まれる文を返す
#やってることはc2iと多分同じアルゴリズム
def answer_find(context_text,answer_start,answer_end,answer_replace):
    context=sent_tokenize(context_text)
    sent_start_id=-1
    sent_end_id=-1
    for i,sent in enumerate(context):
        current_p=context_text.find(sent)
        end_p=current_p+len(sent)
        if current_p<=answer_start and sent_start_id==-1:
            sent_start_id=i
        if current_p<=answer_end and sent_end_id==-1:
            sent_end_id=i
    if sent_start_id==-1 or sent_end_id==-1:
        sys.exit(-1)
    answer_sent=" ".join(context[sent_start_id:sent_end_id+1])
    #ここで答えを置換する方法。ピリオドが消滅した場合などに危険なので止める。
    """
    if answer_replace:
        context_text=context_text.replace(context[answer_start:answer_end],"<answer_word>")
        answer_sent=sent_tokenize(context_text)[sent_start_id]
    """
    return answer_sent,sent_start_id



def data_process(input_path,src_path,tgt_path,question_modify,train=True,sub=False):
    with open(input_path,"r") as f:
        data=json.load(f)
    contexts=[]
    questions=[]
    answer_starts=[]
    answer_ends=[]
    answer_texts=[]
    answers=[]
    sentences=[]
    ids=[]
    answer_replace=False
    cq_list=[]
    for paragraph in tqdm(data["data"]):
        context_text=paragraph["story"].lower()
        question_history=[]
        for i in range(len(paragraph["questions"])):
            question_dict=paragraph["questions"][i]
            answer_dict=paragraph["answers"][i]
            question_text=question_dict["input_text"]
            answer_text=answer_dict["input_text"]
            question_history.append(question_text)

            span_start=answer_dict["span_start"]
            span_end=answer_dict["span_end"]
            span_text=answer_dict["span_text"]
            turn_id=paragraph["questions"][i]["turn_id"]
            if span_start==-1:
                continue
            sentence,answer_index=answer_find(context_text,span_start,span_end,answer_replace)
            sentence=modify(sentence,question_text,answer_text,answer_replace)
            sentence=" ".join(tokenize(sentence))
            context_text=" ".join(tokenize(context_text))
            question_text=" ".join(tokenize(question_text))
            cq_list.append((context_text,question_text,answer_index,question_history))
    return cq_list
                


#main
version="1.1"
type="normal"
question_modify=True
question_interro=True


cq_list=data_process(input_path="data/coqa-dev.json",
            src_path="data/coqa-src-dev-{}.txt".format(type),
            tgt_path="data/coqa-tgt-dev-{}.txt".format(type),
            question_modify=True,
            train=False,
            sub=True
            )




  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 2/500 [00:00<00:29, 17.00it/s]

  1%|          | 3/500 [00:00<00:40, 12.13it/s]

  1%|          | 5/500 [00:00<00:40, 12.30it/s]

  1%|▏         | 7/500 [00:00<00:39, 12.52it/s]

  2%|▏         | 8/500 [00:00<00:42, 11.55it/s]

  2%|▏         | 9/500 [00:00<00:43, 11.17it/s]

  2%|▏         | 12/500 [00:00<00:38, 12.52it/s]

  3%|▎         | 14/500 [00:01<00:37, 12.94it/s]

  3%|▎         | 16/500 [00:01<00:37, 12.78it/s]

  4%|▎         | 18/500 [00:01<00:38, 12.67it/s]

  4%|▍         | 20/500 [00:01<00:36, 13.06it/s]

  4%|▍         | 22/500 [00:01<00:36, 13.01it/s]

  5%|▍         | 24/500 [00:01<00:36, 13.16it/s]

  5%|▌         | 26/500 [00:02<00:37, 12.66it/s]

  6%|▌         | 28/500 [00:02<00:37, 12.74it/s]

  6%|▌         | 30/500 [00:02<00:36, 12.88it/s]

  6%|▋         | 32/500 [00:02<00:35, 13.01it/s]

  7%|▋         | 34/500 [00:02<00:37, 12.58it/s]

  7%|▋         | 36/500 [00:02<00:37, 12.23it/s]

  8%|▊       

 59%|█████▉    | 294/500 [00:22<00:15, 13.10it/s]

 59%|█████▉    | 296/500 [00:22<00:15, 13.10it/s]

 60%|█████▉    | 299/500 [00:22<00:15, 13.16it/s]

 60%|██████    | 301/500 [00:22<00:15, 13.16it/s]

 61%|██████    | 303/500 [00:22<00:14, 13.17it/s]

 61%|██████    | 305/500 [00:23<00:14, 13.13it/s]

 61%|██████▏   | 307/500 [00:23<00:14, 13.14it/s]

 62%|██████▏   | 309/500 [00:23<00:14, 13.17it/s]

 62%|██████▏   | 311/500 [00:23<00:14, 13.16it/s]

 63%|██████▎   | 313/500 [00:23<00:14, 13.16it/s]

 63%|██████▎   | 315/500 [00:23<00:14, 13.15it/s]

 63%|██████▎   | 317/500 [00:24<00:13, 13.14it/s]

 64%|██████▍   | 320/500 [00:24<00:13, 13.20it/s]

 64%|██████▍   | 322/500 [00:24<00:13, 13.18it/s]

 65%|██████▍   | 324/500 [00:24<00:13, 13.18it/s]

 65%|██████▌   | 326/500 [00:24<00:13, 13.20it/s]

 66%|██████▌   | 328/500 [00:24<00:13, 13.21it/s]

 66%|██████▌   | 330/500 [00:24<00:12, 13.22it/s]

 66%|██████▋   | 332/500 [00:25<00:12, 13.25it/s]

 67%|██████▋   | 334/500 [00:25

In [126]:
a="aa"
b="bb"
c=" ".join([a,b])
print(a,b,c)

aa bb aa bb


In [144]:
print(cq_list[0],len(cq_list[0]))

('once upon a time , in a barn near a farm house , there lived a little white kitten named cotton . cotton lived high up in a nice warm place above the barn where all of the farmer \'s horses slept . but cotton was n\'t alone in her little home above the barn , oh no . she shared her hay bed with her mommy and 5 other sisters . all of her sisters were cute and fluffy , like cotton . but she was the only white one in the bunch . the rest of her sisters were all orange with beautiful white tiger stripes like cotton \'s mommy . being different made cotton quite sad . she often wished she looked like the rest of her family . so one day , when cotton found a can of the old farmer \'s orange paint , she used it to paint herself like them . when her mommy and sisters found her they started laughing . " what are you doing , cotton ? ! " " i only wanted to be more like you " . cotton \'s mommy rubbed her face on cotton \'s and said " oh cotton , but your fur is so pretty and special , like you 

In [131]:
for cq in cq_list[0:100]:
    print(cq,len(cq))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [153]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize,sent_tokenize
from tqdm import tqdm

def TF_IDF(corpus_text):

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus_text)
    return X.toarray()

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    

def sent_retrieve(paragraph,question):
    paragraph=sent_tokenize(paragraph)
    paragraph.append(question)
    mat=TF_IDF(paragraph)
    cos_sim_list=[cos_sim(m,mat[-1]) for m in mat[:-1]]
    index=np.argmax(cos_sim_list)
    return index
    

count=0
length=1000
for i in tqdm(range(length)):
    history=" ".join(cq_list[i][3][-2:])
    if sent_retrieve(cq_list[i][0],history)==cq_list[i][2]:
        count+=1
        
print(count/length)

    

    



  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


  1%|          | 11/1000 [00:00<00:09, 101.12it/s]

  2%|▏         | 17/1000 [00:00<00:12, 81.52it/s] 

  2%|▏         | 24/1000 [00:00<00:12, 77.47it/s]

  3%|▎         | 29/1000 [00:00<00:15, 63.67it/s]

  4%|▎         | 37/1000 [00:00<00:14, 66.27it/s]

  4%|▍         | 45/1000 [00:00<00:14, 67.78it/s]

  5%|▌         | 52/1000 [00:00<00:14, 64.23it/s]

  6%|▌         | 58/1000 [00:00<00:15, 62.12it/s]

  6%|▋         | 65/1000 [00:01<00:14, 62.41it/s]

  7%|▋         | 74/1000 [00:01<00:14, 63.90it/s]

  8%|▊         | 83/1000 [00:01<00:13, 65.63it/s]

  9%|▉         | 90/1000 [00:01<00:14, 64.25it/s]

 10%|▉         | 97/1000 [00:01<00:14, 62.33it/s]

 11%|█         | 108/1000 [00:01<00:13, 64.92it/s]

 12%|█▏        | 118/1000 [00:01<00:13, 66.63it/s]

 13%|█▎        | 126/1000 [00:01<00:13, 64.84it/s]

 13%|█▎

0.02


In [149]:
aa=[3,4]
print(aa[-3:])

[3, 4]


In [140]:
print([len(sent_tokenize(cq[0])) for cq in cq_list[5600:5800]])

[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]


In [137]:
count=sum([len(sent_tokenize(cq[0])) for cq in cq_list])
print(count/len(cq_list))

16.34592068704218


In [21]:


x=[["a","b","c"],["a","b","c","d"]]
corpus=["I have a dog.","I have one dog."]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.toarray())

[[0.70710678 0.70710678 0.        ]
 [0.50154891 0.50154891 0.70490949]]
